In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap
from qiskit.visualization import plot_histogram, plot_distribution
from qiskit.quantum_info import SparsePauliOp

from qiskit_machine_learning.neural_networks import EstimatorQNN, SamplerQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals
from qiskit.primitives import Sampler, Estimator

from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Method to construct the Convolutional Layer for the QCNN
def conv_layer(qc:QuantumCircuit(), para:ParameterVector, iter:int) -> QuantumCircuit():
    start = 0 if qc.num_qubits == 2 else -1
    iter = 3 if iter == 0 else iter
    for i in range(start, qc.num_qubits-1):
        qc.rz(-np.pi / 2, i+1)
        qc.cx(i+1, i)
        qc.rz(para[(3*i)+iter], i)
        qc.ry(para[(3*i)+1+iter], i+1)
        qc.cx(i, i+1)
        qc.ry(para[(3*i)+2+iter], i+1)
        qc.cx(i+1, i)
        qc.rz(np.pi / 2, i)
        qc.barrier()
    if iter == 3:
        iter = 0
    if qc.num_qubits == 2:
        iter += 3
    else: 
        iter += (3*(qc.num_qubits)) 
    return qc, iter

# Method to construct the Pooling Layer for the QCNN
def pool_layer(qc:QuantumCircuit(), para:ParameterVector, iter:int) -> QuantumCircuit():
    half = int(qc.num_qubits/2)
    for i in range(half):
        qc.rz(-np.pi / 2, i+half)
        qc.cx(i+half, i)
        qc.rz(para[(3*i)+iter], i)
        qc.ry(para[(3*i)+1+iter], i+half)
        qc.cx(i, i+half)
        qc.ry(para[(3*i)+2+iter], i+half)
        qc.barrier()
    iter += (half*3)
    return qc, iter

# Method that builds the entire circuit and layers systematically
def qcnn_circuit(num_qubits:int, num_outputs:int) -> QuantumCircuit():
    qc = QuantumCircuit(num_qubits)
    num_outputs *= 2
    num_layers = np.log2(num_qubits) # Calculates number of convolutional/pooling layers
    
    feature_map = ZFeatureMap(num_qubits)  # Pulls in Qiskit's built in ZFeatureMap
    qc.compose(feature_map, range(num_qubits), inplace=True) # Adds ZFeatureMap to circuit for initialization
    
    parameter_vec = ParameterVector(name="θ", length=int(9*(2**num_layers))-12) # Calculates number of parameters needed
   
    iter = 0 # Tracks where in the Parameter Vector to put the next parameter
    index = num_qubits # Divides itself in half at each layer
    layer = 1 # Tracks the current layer being constructed
    while index > int(num_outputs): # Adds 1 convolution layer and 1 pooling layer then iterates until the desired amount of outputs remain
        conv_qc = QuantumCircuit(index)
        pool_qc = QuantumCircuit(index)
        
        # Adds 1 convolutional layer to the circuit by converting the convolutional circuit to a gate
        conv_qc, iter = conv_layer(conv_qc, parameter_vec, iter)
        qc.compose(conv_qc.to_instruction(label=f'Convolution Layer {layer}'), range(int(num_qubits - index), num_qubits), inplace=True)
        
        # Adds 1 pooling layer to the circuit by converting the pooling circuit to a gate
        pool_qc, iter = pool_layer(pool_qc, parameter_vec, iter)
        qc.compose(pool_qc.to_instruction(label=f'Pooling Layer {layer}'), range(int(num_qubits - index), num_qubits), inplace=True)

        # Handle tracker increment or decrement
        index = int(index / 2)
        layer += 1

    # Constructs weight parameter by removing ZFeatureMap's parameters from the total
    weight_params = qc.parameters - feature_map.parameters
    return qc, feature_map.parameters, weight_params

**importing training data**

In [3]:
from zipfile import ZipFile

In [4]:
# specifying the zip file name 
file_name = "seeds.zip"
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    zip.extractall() 

File Name                                             Modified             Size
seeds_dataset.txt                              2023-05-22 15:21:46         9300


In [5]:
#loading data
data = np.genfromtxt("sonar.all-data", delimiter=',', dtype=str)

#splitting the data to features and labels
features = data[:, :-1].astype(float)  
labels = data[:, -1]

print("Features shape:", features.shape)
print("Labels shape:", labels.shape)

features_list = [np.array(row) for row in features]
features_list = [row[:16] for row in features_list]

Features shape: (208, 60)
Labels shape: (208,)


In [ ]:
in_qubits, out_qubits = 16, 2
qcnn2, input_params2, weight_params2 = qcnn_circuit(in_qubits, out_qubits)

def func(x):
    return x % 2**out_qubits # Mod amount must match out_qubits

qcnn_est2 = SamplerQNN(circuit=qcnn2.decompose(),
                       input_params=input_params2, weight_params=weight_params2,
                       interpret=func, output_shape=(2**out_qubits))
classifier2 = NeuralNetworkClassifier(qcnn_est2, optimizer=COBYLA(maxiter=500))
print(qcnn_est2.output_shape)

trainer, tester, trainer_ans, tester_ans = train_test_split(features_list, labels, test_size=0.25, random_state=246)

x2 = np.asarray(trainer)
y2 = np.asarray(trainer_ans)

objective_func_vals2 = []

plt.rcParams["figure.figsize"] = (12, 6)

In [ ]:
classifier2.fit(x2, y2)